In [8]:


import os
import sys

import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F  
import matplotlib 
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
import torch.utils.data
from torch.utils.data import DataLoader
import numpy as np
from scipy.signal import kaiserord, firwin, lfilter
import gzip
import pickle
import scipy.signal as signal
import math
import torch
import random

sys.path.append("/home/jfeng/Desktop/jfeng/rf_spoofing/spoofing/models")


def apply_iq_imbalances(x, G_I, G_Q, phi, dc_I, dc_Q):
    I = x.real
    Q = x.imag
    ''' 
    ratio = G_Q / G_I
    G_I_ratio = np.sqrt(2 / (1 + ratio**2))
    G_Q_ratio = ratio * G_I
    I_g = G_I_ratio * I
    Q_g = G_Q_ratio * Q

    cos_phi = np.cos(phi)
    tan_phi = np.tan(phi)
    Q_phi = Q_g / cos_phi + I_g * tan_phi
    I_phi = I_g
    I_final = I_phi + dc_I
    Q_final = Q_phi + dc_Q
    '''
    # DC Offset
    I_offset = I + dc_I
    Q_offset = Q + dc_Q
    
    # Gain
    gain_ratio = G_Q / G_I
    I_gain = I_offset
    Q_gain = Q_offset * gain_ratio
    
    # Phase
    I_final = I_gain
    Q_final = Q_gain * np.cos(phi) + I_gain * np.sin(phi)

    x_out = I_final + Q_final * 1j
    
    return x_out

### Transmitter Data ###
FILENAME_TX = f"/home/jfeng/Desktop/May27/Pluto_10_test.iq"

print("Current Transmitter File Processing: ", FILENAME_TX)
# Read in the data
data_tx = np.fromfile(FILENAME_TX, dtype="float32")
# Bring I and Q into one value
I = data_tx[0::2]
Q = data_tx[1::2]
data_complex_tx = I + 1j*Q

### Adversary Noise Data ###
FILENAME_ADV_TX = f"May27/goodParams.iq"
print("Current Adversary Noise File Processing: ", FILENAME_ADV_TX)
# Read in the data
data_adv_tx = np.fromfile(FILENAME_ADV_TX, dtype="float32")
# Bring I and Q into one value
I = data_adv_tx[0::2]
Q = data_adv_tx[1::2]
data_complex_adv_tx = I + 1j*Q

data_complex_adv_tx = data_complex_adv_tx
for i in range (0, len(data_complex_adv_tx)):
    if (abs(data_complex_adv_tx[i]) > 1):
        print("signal strength too high, redo")

# Adjust
# Path loss exponent
n = 2
# Reference distance
d1 = 1 
# Actual distance
d2 = 8


dc_offset_I_list = [-0.0000048145, -0.0000050625, -0.0000049675, -0.0000049891, -0.0000050236, -0.0000049773, -0.0000052696, -0.0000050665, -0.0000049802, -0.0000050692]
dc_offset_Q_list = [-0.0000048619, -0.0000049788, -0.0000049042, -0.0000048843, -0.0000048275, -0.0000051674, -0.0000050298, -0.0000046535, -0.0000049399, -0.0000049609]
gain_I_list = [0.04946, 0.02505, 0.00776, 0.00737, 0.03989, 0.04072, 0.04677, 0.06374, 0.01650, 0.01540]
gain_Q_list = [0.04948, 0.02504, 0.00775, 0.00737, 0.03991, 0.04074, 0.04675, 0.06376, 0.01649, 0.01540]
phase_imbalance_list = [-0.00058, 0.00060, -0.00013, 0.00025, -0.00045, -0.00078, 0.00066, -0.00046, 0.00029, -0.00044]

random_index = random.randint(0, 9)
dc_offset_I = dc_offset_I_list[random_index]
dc_offset_Q = dc_offset_Q_list[random_index]
gain_I = gain_I_list[random_index]
gain_Q = gain_Q_list[random_index]
phase_imbalance = phase_imbalance_list[random_index]

data_complex_adv_tx = data_complex_adv_tx / 0.30
x_imbalance = apply_iq_imbalances(data_complex_adv_tx, gain_I, gain_Q, phase_imbalance, dc_offset_I, dc_offset_Q)
#x_imbalance = apply_iq_imbalances(data_complex_adv_tx, 0.04, 0.04002, -0.00004, -0.000005, -0.000005) 

# Path loss
scaling = (d1 / d2) ** (n / 2)
iq_path = x_imbalance * scaling * 0.25
print("Scaled signal: ", iq_path)
# Rician fading
k = 10.0

i_window = iq_path.real
q_window = iq_path.imag
scale_LOS = np.sqrt(k / (k + 1))
scale_NLOS = np.sqrt(1 / (k + 1))
real_nlos = np.random.randn(*i_window.shape) / np.sqrt(2.0)
imag_nlos = np.random.randn(*i_window.shape) / np.sqrt(2.0)
real_los = np.ones(i_window.shape)
imag_los = np.zeros(i_window.shape)
real_fade = scale_LOS * real_los + scale_NLOS * real_nlos
imag_fade = scale_LOS * imag_los + scale_NLOS * imag_nlos
faded_real = i_window * real_fade - q_window * imag_fade
faded_imag = i_window * imag_fade + q_window * real_fade
iq_scaled = faded_real + 1j*faded_imag

noise_std = 0.001

# AWGN noise
per_dim_std = noise_std / np.sqrt(2.0)
noise = np.random.randn(*iq_scaled.shape) * per_dim_std
iq_received = iq_scaled + noise

### Combine Datas ###
len_data_complex_tx = len(data_complex_tx)
print("Length of transmitter: ", len_data_complex_tx)
len_data_complex_adv_tx = len(data_complex_adv_tx)
print("Length of noise: ", len_data_complex_adv_tx)
num_chunks = len_data_complex_tx // len_data_complex_adv_tx
combined_array = data_complex_tx.copy()
for i in range(num_chunks):
    start = i * len_data_complex_adv_tx
    end = start + len_data_complex_adv_tx
    combined_array[start:end] = data_complex_tx[start:end] + iq_received

print("Original Tx: ", data_complex_tx)
print("Noise: ", data_complex_adv_tx)
print("Path loss noise: ", iq_path)
print("Rician noise: ", iq_scaled)
print("With noise: ", iq_received)
print("Combined: ", combined_array)
real = combined_array.real
imag = combined_array.imag
print("Mean Magnitude tx: ", np.mean(np.abs(data_complex_tx)))
print("Mean Magnitude Noise: ", np.mean(np.abs(data_complex_adv_tx)))
#print("Mean Magnitude after imbalance: ", np.mean(np.abs(x_imbalance)))
print("Mean Magnitude noise after transformations: ", np.mean(np.abs(iq_received)))
print("Mean Magnitude combined: ", np.mean(np.abs(real + imag * 1j)))
data_to_save = [elem for pair in zip(real, imag) for elem in pair]
data_to_save = np.array(data_to_save)
float32_data_to_save = data_to_save.astype("float32")
float32_data_to_save.tofile("May30/testbed/ota_output.iq")

import torch
import torch.utils
import torch.utils.data
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset
from torchsummary import summary
from attempt2 import resnet50_1d
import gzip
import tqdm
import os
#from models.cdcn import cdcn

class IQDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        sample = self.data[idx]
        label = self.labels[idx]

        sample = torch.from_numpy(sample).float()
        # Normalize data
        magnitude = torch.sqrt(torch.sum(sample**2, dim=1, keepdim=True))
        sample = sample / magnitude

        label_tensors = torch.tensor(label, dtype=torch.long)

        return sample, label_tensors


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Using {} device'.format(device))

folder_path = '/home/jfeng/Desktop/May23'
for filename in os.listdir(folder_path):
    test_data_tensors = []
    test_label_tensors = []
    file_path = os.path.join(folder_path, filename)
    #print(f"File: {file_path}")
    file_name = f"May30/testbed/ota_output.iq"
    target = 2
    data = np.fromfile(file_name, dtype="float32")
    size_data = len(data)
    #print("Size of data: ", size_data)
    size_half = int(size_data / 2)
    real_part = data[0::2]
    #print("Real Length: ", len(real_part))
    imag_part = data[1::2]
    #print("Imag Length: ", len(imag_part))
    print("Mean Magnitude: ", np.mean(np.abs(real_part + imag_part * 1j)))
    #print(real_part[5000])
    #print(real_part[0])
    #print(imag_part[5000])
    #print(imag_part[0])

    for x in range(4800, 6000):
        # Implement a 10% hop
        combined_data = np.vstack((real_part[(x+1)*500:(x+1)*500+10000], imag_part[(x+1)*500:(x+1)*500+10000]))
        array = np.zeros(combined_data.shape[1])
        array.fill(target)
        test_data_tensors.append(combined_data)
        test_label_tensors.append(target)

    # Format the data
    test_data_tensors = np.stack(test_data_tensors, axis=0)
    test_label_tensors = np.array(test_label_tensors)

    # Randomize the data
    indices = np.random.permutation(len(test_data_tensors))
    test_data = test_data_tensors[indices]
    test_labels = test_label_tensors[indices]
    #print(test_data.shape)
    #print(test_labels.shape)

    # Create the dataset
    test_dataset = IQDataset(test_data, test_labels)

    batch_size = 16

    # Load the data
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    torch.manual_seed(7)
    np.random.seed(7)

    device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
    #print(device)
    net = resnet50_1d(num_classes=8)
    net.to(device)
    net.train()
    #summary(net, input_size = (1, 2), batch_size = 8)


    dataiter = iter(test_loader)
    images, labels = next(dataiter)

    PATH = "/home/jfeng/Desktop/jfeng/rf_spoofing/spoofing/weights/best_model_retrained.pth"

    #net = cdcn(num_classes=8)
    net = resnet50_1d(num_classes=8)
    #net.load_state_dict(torch.load(PATH))
    net.load_state_dict(torch.load(PATH, map_location=torch.device("cpu")))
    net.to(device)
    net.eval()

    correct = 0
    total = 0
    conf_threshold = 0.85
    fooled = 0
    confidence = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            
            #for x in range(0, len(predicted)):
                #print(predicted)
                #print(labels)
            #    if ~(predicted[x] == labels[x]):
            #        print("Predicted: ", predicted[x])
            #        print("Label: ", labels[x])
            correct += (predicted == labels).sum().item()
            probs = torch.nn.functional.softmax(outputs, dim=1)
            conf, classes = torch.max(probs, 1)
           
            print(predicted)
            # For GAN
            counter = 0
            for conf_i in conf:
                # For pgd attack
                #if conf_i > conf_threshold or classes[counter] == target and classes[counter] != 1:
                # For testing normal mode
                if conf_i > conf_threshold and classes[counter] == target:
                # For generative attack
                #if (conf_i > conf_threshold):
                    fooled = fooled + 1
                    confidence = confidence + conf_i
                counter = counter + 1
            
            # For EOT
            '''
            for x in range(0, len(predicted)):
                print(predicted)
                if (predicted[x] == 3):
                    fooled = fooled + 1
                    confidence = confidence + conf[x]
            '''

    ### For fingerprinter ###
    acc  = correct / total * 100## stores the accuracy computed in the above loop
    print('Accuracy of the network on the 10000 test images: %d %%' % (acc))

    ### For Attack ###
    fooled_acc = fooled / total * 100
    if (fooled == 0):
        print("No fools")
    else:
        fooled_conf = confidence / fooled * 100
        print("Average confidence: ", fooled_conf)
    print('Percentage of successful fools: %d %%' % (fooled_acc), " , Number fooled: ", fooled, " , Total: ", total)
    #print("Number fooled: ", fooled)
    #print("Number total: ", total)
    break



Current Transmitter File Processing:  /home/jfeng/Desktop/May27/Pluto_10_test.iq
Current Adversary Noise File Processing:  May27/goodParams.iq
Scaled signal:  [ 0.0063769 +0.00361064j  0.01041508-0.00076242j -0.00852758-0.00498366j
 ...  0.00556972-0.00659108j  0.01011588-0.00253616j
 -0.0081386 -0.00835786j]
Length of transmitter:  41624160
Length of noise:  10000
Original Tx:  [-0.00051881-2.7466653e-04j  0.00070193+3.0518502e-05j
  0.00067141+6.7140703e-04j ...  0.00057985+4.8829603e-04j
  0.00079348+3.3570352e-04j  0.00015259+3.0518502e-05j]
Noise:  [ 0.20406568+0.11572116j  0.3332876 -0.02438071j -0.2728776 -0.1597135j
 ...  0.17823613-0.21115874j  0.3237131 -0.08121492j
 -0.26043016-0.26782575j]
Path loss noise:  [ 0.0063769 +0.00361064j  0.01041508-0.00076242j -0.00852758-0.00498366j
 ...  0.00556972-0.00659108j  0.01011588-0.00253616j
 -0.0081386 -0.00835786j]
Rician noise:  [ 0.00456925+0.00284332j  0.00665918-0.00202765j -0.0065408 -0.00513567j
 ...  0.00495276-0.0051572j   0

/tmp/ipykernel_3895394/2667156736.py:278: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load(PATH, map_location=torch.device("cpu")))


tensor([1, 0, 0, 0, 0, 0, 0, 4, 4, 4, 4, 0, 0, 0, 0, 0], device='cuda:1')
tensor([0, 6, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3], device='cuda:1')
tensor([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:1')
tensor([0, 0, 4, 0, 2, 0, 0, 1, 0, 0, 0, 4, 0, 0, 0, 0], device='cuda:1')
tensor([0, 3, 0, 0, 0, 0, 4, 0, 0, 1, 0, 0, 0, 1, 0, 0], device='cuda:1')
tensor([0, 0, 5, 0, 0, 1, 0, 4, 0, 0, 0, 0, 4, 0, 0, 0], device='cuda:1')
tensor([7, 4, 5, 0, 0, 0, 0, 2, 4, 1, 5, 0, 0, 5, 2, 0], device='cuda:1')
tensor([0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0], device='cuda:1')
tensor([1, 4, 1, 0, 5, 4, 4, 0, 0, 0, 5, 0, 0, 4, 0, 0], device='cuda:1')
tensor([0, 4, 0, 5, 4, 0, 0, 5, 0, 0, 0, 0, 5, 0, 4, 0], device='cuda:1')
tensor([0, 5, 4, 0, 0, 0, 0, 0, 0, 0, 1, 4, 0, 1, 0, 4], device='cuda:1')
tensor([0, 0, 0, 0, 0, 0, 0, 0, 2, 4, 4, 0, 5, 0, 0, 4], device='cuda:1')
tensor([6, 0, 0, 4, 0, 0, 1, 0, 0, 0, 0, 5, 0, 0, 0, 0], device='cuda:1')
tensor([0, 0, 0, 3, 0, 0, 0, 0, 0, 4, 